In [1]:
from base import *


In [2]:
comment_data = pd.read_csv('data/JData_Comment.csv')
len(comment_data)

558552

In [3]:
comment_array = comment_data.drop('dt', 1).as_matrix()
nrow = comment_data.groupby('dt')['sku_id'].count()[0]
ncol = comment_array.shape[1] - 1
num_section = comment_array.shape[0] / nrow
stashed_array = np.zeros([nrow, ncol * num_section + 1])
columns = np.array(comment_data.columns.drop(['dt', 'sku_id']))
colnames = np.array([' ' * 20] * (ncol * num_section + 1))
colnames[0] = 'sku_id'
stashed_array[:, 0] = comment_array[:nrow, 0]
for nsec in range(num_section):
    assert np.sum(stashed_array[:, 0] != comment_array[nsec * nrow:(nsec + 1) * nrow, 0]) == 0
    stashed_array[:, 1 + nsec * ncol:(nsec + 1) * ncol + 1] = comment_array[nsec * nrow:(nsec + 1) * nrow, 1:]
    colnames[1 + nsec * ncol:(nsec + 1) * ncol + 1] = columns + str(nsec)
    


In [7]:
comments = pd.DataFrame(stashed_array, columns=colnames)
comments['sku_id'] = comments['sku_id'].astype(int)
comments = comments.set_index('sku_id')
com_row = pd.DataFrame([[np.nan] * len(colnames)], columns=colnames)
comments = comments.append(com_row)
comments.tail()

,bad_comment_rate0,bad_comment_rate1,bad_comment_rate10,bad_comment_rate11,bad_comment_rate2,bad_comment_rate3,bad_comment_rate4,bad_comment_rate5,bad_comment_rate6,bad_comment_rate7,...,has_bad_comment11,has_bad_comment2,has_bad_comment3,has_bad_comment4,has_bad_comment5,has_bad_comment6,has_bad_comment7,has_bad_comment8,has_bad_comment9,sku_id
99988,0.0488,0.0488,0.0488,0.0488,0.0488,0.0488,0.0488,0.0488,0.0488,0.0488,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
9999,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
99993,0.0669,0.0670,0.0647,0.0645,0.0667,0.0669,0.0664,0.0658,0.0653,0.0653,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
99998,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
action_all['time'] = pd.to_datetime(action_all['time']) - pd.to_datetime(test_start)
action_all['time'] = action_all['time'].apply(lambda x:x.total_seconds())
tmp_frame = pd.DataFrame(action_all['sku_id'])

In [9]:
comment_times = pd.to_datetime(comment_data['dt'].unique()) - pd.to_datetime(test_start)
comment_times = np.array([x.total_seconds() for x in comment_times])
comment_interval = comment_times[1] - comment_times[0]


In [10]:
def get_comment_data(column_name):
    for sec_num in range(len(comment_times)):
        tmp_frame['bflag' + str(sec_num)] = np.array(np.abs(action_all['time'] - 
                                                         comment_times[sec_num]) < comment_interval / 2,
                                                  dtype=float)
#    tmp_frame['bflag0'] = tmp_frame['bflag0'] + np.array(action_all['time'] <= comment_interval / 2, dtype=float)
    last_flag = 'bflag' + str(len(comment_times) - 1) 
    tmp_frame[last_flag] = np.array(action_all['time'] >= comment_times[-2] + comment_interval / 2,
                                                            dtype=float)
    final_values = np.zeros(len(tmp_frame))
    sku_ids = np.array(action_all['sku_id'])
    sku_ids[np.logical_not(np.in1d(sku_ids, comments.index))] = 0
    for sec_num in range(len(comment_times)):
        values = comments.loc[action_all['sku_id'], column_name + str(sec_num)]
        final_values += values * tmp_frame['bflag' + str(sec_num)]
        
    return final_values


action_all['sku_comments'] = get_comment_data('bad_comment_rate')
